<a href="https://colab.research.google.com/github/peppefdf/CSL_Gipuzkoa/blob/main/Dash_interactive_IsochronicMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
#%pip install osmnx
#%pip install dash
#%pip install dash_leaflet
#%pip install dash_bootstrap_components

import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash import Dash, html, dcc, Output, Input, State, callback
import dash_leaflet as dl
from shapely.geometry import Polygon, mapping

import osmnx as ox
import networkx as nx
from shapely.geometry import Point
import json
import geopandas as gpd

center = [43.27310034205181,-2.019703226103527]

app = dash.Dash(external_stylesheets=[dbc.themes.FLATLY])

sidebar = html.Div(
    [
      html.P([ html.Br(),'Choose transport mode']),
      dcc.Dropdown(['walk', 'bike', 'drive'], id='dropdown_TransOpt'),
      html.P([ html.Br(),'Time for Isochronic Map (in min)']),
      dcc.Input(id="choose_time", type="text", placeholder="", style={'marginRight':'10px','width': 50,'height': 20}),
      html.Div(id='out_text')
    ]
)

content = html.Div([
    dl.Map([dl.TileLayer(),
           dl.Polygon(positions=[], id='position-data'),
           dl.ScaleControl(position="bottomright")
           ],
           id='mapa',
           center=center, zoom=14, style={'height': '50vh'}
           )
    ])

app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(sidebar, width=3, className='bg-light'),
                dbc.Col(content, width=9)
                ],
            style={"height": "100vh"}
            ),
    ],
    fluid=True
)

#@app.callback(Output('position-data', 'positions'),
#@app.callback(Output('out_text', 'children'),
#               State('dropdown_TransOpt', 'value'),
@app.callback(Output('position-data', 'positions'),
              Input('mapa', 'clickData'),
              State('dropdown_TransOpt', 'value'),  # State does not trigger the callback
              State('choose_time', 'value'),
              prevent_initial_call=True)
def on_click(coord,opt,t):
    #try:
    if coord is not None:
        Lat = coord['latlng']['lat']
        Lon = coord['latlng']['lng']
        map_center = (Lat, Lon)
        distance = 1000
        #walk_times = [5,10,15] # in minutes
        walk_times = [float(t)]
        G = ox.graph_from_point(map_center, dist=distance, network_type=opt, simplify=False)
        if opt == 'bike':
           travel_speed = 20 # km/hour
           meters_per_minute = travel_speed * 1000 / 60 #km per hour to m per minute
           for u, v, k, data in G.edges(data=True, keys=True):
               data['time'] = data['length'] / meters_per_minute

        if opt == 'walk':
           travel_speed = 4.5 # km/hour
           meters_per_minute = travel_speed * 1000 / 60 #km per hour to m per minute
           for u, v, k, data in G.edges(data=True, keys=True):
               data['time'] = data['length'] / meters_per_minute

        if opt == 'drive':
           # fill in edges with missing `maxspeed` from OSM (km/hour)
           hwy_speeds = {"residential": 35, "secondary": 50, "tertiary": 60}
           G = ox.add_edge_speeds(G, hwy_speeds)
           G = ox.add_edge_travel_times(G)
           for u, v, k, data in G.edges(data=True, keys=True):
               #data['time'] = data['length'] / (data['maxspeed'] *1000 / 60)
               data['time'] = data['length'] / (data['speed_kph'] *1000 / 60)

        # add an edge attribute for time in minutes required to traverse each edge
        # get closes graph nodes to origin and destination
        orig_node = ox.nearest_nodes(G, map_center[1], map_center[0])
        #polys = []
        #for walk_time in walk_times:
        subgraph    = nx.ego_graph(G, orig_node, radius=walk_times[0], distance='time')
        node_points = [Point(data["x"], data["y"]) for node, data in subgraph.nodes(data=True)]
        #node_points = [[data["y"],data["x"]] for node, data in subgraph.nodes(data=True)]
        convhull = gpd.GeoSeries(node_points).unary_union.convex_hull
        poly_mapped = mapping(convhull)
        poly_coordinates = poly_mapped['coordinates'][0]
        poly = [ [coords[1], coords[0]] for coords in poly_coordinates]
        #polys.append(poly)
        #return polys[0]
        return poly
        #return json.dumps(polys[0])
        #return json.dumps(poly)
        #return json.dumps(walk_time)
        #return json.dumps(t)

    else:
       #except:
       return {}
       #return json.dumps('something is wrong...')

if __name__ == "__main__":
    app.run_server(debug=True)

<IPython.core.display.Javascript object>